In [ ]:
import requests
import json
from fileinput import close
import io

class ClovaSpeechClient:
    # Clova Speech invoke URL
    invoke_url = 'https://clovaspeech-gw.ncloud.com/external/v1/2515/45ad113fe857520d01f7debdce76f2fa0468fefa7b13ff091beb960ce2950efa'
    # Clova Speech secret key
    secret = '395be196a5f949d9a9b375cb0702fd55'

    def req_url(self, url, completion, callback=None, userdata=None, forbiddens=None, boostings=None, wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'url': url,
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/url',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_object_storage(self, data_key, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                           wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'dataKey': data_key,
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/object-storage',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_upload(self, file, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                   wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        print(json.dumps(request_body, ensure_ascii=False).encode('UTF-8'))
        files = {
            'media': open(file, 'rb'),
            'params': (None, json.dumps(request_body, ensure_ascii=False).encode('UTF-8'), 'application/json')
        }
        response = requests.post(headers=headers, url=self.invoke_url + '/recognizer/upload', files=files)
        return response

if __name__ == '__main__':
    # res = ClovaSpeechClient().req_url(url='http://example.com/media.mp3', completion='sync')
    # res = ClovaSpeechClient().req_object_storage(data_key='data/media.mp3', completion='sync')
    res = ClovaSpeechClient().req_upload(file='C:/Users/jiki9/clova/data/media.mp4', completion='sync')
    dt = res.json()

b'{"language": "en-US", "completion": "sync", "callback": null, "userdata": null, "wordAlignment": true, "fullText": true, "forbiddens": null, "boostings": null, "diarization": null}'


In [ ]:
import cv2
import sys
import numpy as np
import moviepy.editor as mp
import os



txt_pth = "subtitle.txt"
video_pth = ".//data//media.mp4"

def subtitle_generator(txt_pth,video_pth):
  cap1 = cv2.VideoCapture(video_pth)
  file = open(txt_pth, "r")

  w = round(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
  h = round(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

  fps2 = cap1.get(cv2.CAP_PROP_FPS)
  fps2 = round(fps2)
  fourcc = cv2.VideoWriter_fourcc(*"mp4v")
  delay2 = round(1000/fps2)

  out = cv2.VideoWriter(".//test.mp4",fourcc,fps2,(w,h)) #출력 동영상(음성x) 저장될 경로



  fps1 = cap1.get(cv2.CAP_PROP_FPS)
  delay2 = round(1000/fps1)


  if not cap1.isOpened() :
      print("video open failed")
      sys.exit()

  while True:
      ret,frame = cap1.read()
      if not ret: 
          break
      org=(0,620)
      font=cv2.FONT_HERSHEY_SIMPLEX #font 설정
      text = file.readline()
      size, BaseLine=cv2.getTextSize(text,font,1,2)
      if size[0] > w:
          text = text.split()
          n = len(text) //2
          text = text[:n] + ['\n'] + text[n:]
          text = ' '.join(text)

      
      y0, dy = 620, 25

      for i, line in enumerate(text.split('\n')):
          y = y0 + i*dy
          cv2.rectangle(frame,(0,y),(org[0]+size[0],y-size[1]),(0,0,255),-1) # 글자 배경 지정 ( 색상 : (255,0,0))
          cv2.putText(frame, line, (0, y ), font, 1,(255,0,0), 2) # 글자 넣기 ( 색상 : (255,0,0))

      cv2.imshow("frame",frame)

      out.write(frame)

      if cv2.waitKey(delay2) == 27:
          break

def combine_audio(video_pth):
  clip = mp.VideoFileClip(video_pth)
  clip.audio.write_audiofile("audio.mp3")
  cmd = "ffmpeg -i {} -i audio.mp3 -c:v copy -c:a aac output.mp4".format(video_pth)
  os.system(cmd)
